In [4]:
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def extract_features(image, detector):
    if image is None:
        return None, None
    keypoints, descriptors = detector.detectAndCompute(image, None)
    return keypoints, descriptors

feature_detector = cv2.SIFT_create()

all_keypoints = []
all_descriptors = []
labels = []

In [5]:

natural_image_paths = os.listdir('./dataset/natural_training')
for path in natural_image_paths:
    image = cv2.imread('./dataset/natural_training/' + path)
    keypoints, descriptors = extract_features(image, feature_detector)
    if descriptors is not None:
        all_keypoints.extend(keypoints)
        all_descriptors.append(descriptors)
        labels.append(0) 

manmade_image_paths = os.listdir('./dataset/manmade_training')
for path in manmade_image_paths:
    image = cv2.imread('./dataset/manmade_training/' + path)
    keypoints, descriptors = extract_features(image, feature_detector)
    if descriptors is not None:
        all_keypoints.extend(keypoints)
        all_descriptors.append(descriptors)
        labels.append(1) 

libpng warning: iCCP: known incorrect sRGB profile


In [ ]:
# cluster all the descriptors
all_descriptors = np.vstack(all_descriptors)

num_clusters = 100
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
kmeans.fit(all_descriptors)

/Users/lumizhang/miniconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KeyboardInterrupt: 

KeyboardInterrupt: 

In [ ]:
histogram = np.zeros(num_clusters)
bof_features = []
for descriptors in all_descriptors:
    cluster_labels = kmeans.predict([descriptors])
    histogram[cluster_labels] += 1
    if len(histogram) == num_clusters:
        bof_features.append(histogram)
        histogram = np.zeros(num_clusters)

scaler = StandardScaler()
bof_features = scaler.fit_transform(bof_features)

In [ ]:
# Train an SVM classifier
svm_classifier = SVC(C=1, kernel='linear', random_state=0)
svm_classifier.fit(bof_features, labels)

In [ ]:

y_pred = svm_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")